In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from __future__ import print_function

import os

import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import numpy as np
import pandas as pd
import time


try:
    import keras
    from imblearn.keras import balanced_batch_generator
    from imblearn.under_sampling import NearMiss
    import keras.backend as K
    from keras.models import Sequential
    from keras.models import model_from_json
    from keras.layers import Dense, Dropout, Activation
    from keras.optimizers import RMSprop
    from keras.optimizers import Adam
    from keras.optimizers import Adagrad
    from keras.optimizers import SGD
    from keras.callbacks import LambdaCallback, ReduceLROnPlateau, ModelCheckpoint
    from keras.layers.core import Lambda
    from keras.losses import categorical_crossentropy
    import tensorflow as tf
    from keras import regularizers

except:
    print("Keras not found")
    
np.random.seed(42)


def my_basename(path):
    return os.path.splitext(os.path.split(path)[1])[0]

file_dir=r'C:/Users/Mark.Rademaker/PycharmProjects/InternshipNaturalis/trait-geo-diverse-dl/concept proof'

Using TensorFlow backend.


In [3]:
#access file with list of taxa names
taxa=pd.read_csv(file_dir+"/data/spec_filtered/taxa.txt",header=None)
taxa.columns=["taxon"] 

In [16]:
pred_loc=pd.read_csv(file_dir + "/data/GIS/world_locations_to_predict.csv")
lon = pred_loc["decimal_longitude"]
lat = pred_loc["decimal_latitude"]
lon = lon.to_numpy()
lat = lat.to_numpy()


#input stacked tiff
src=rasterio.open(file_dir+'/data/GIS/env_stacked/ENVIREM_BIOCLIM_stacked.tif')
profile=src.profile
band=src.read(1) #as np array


#for species in taxa["taxon"][1:]:
spec="Alcelaphus_buselaphus"
new_band=band.copy() # to write results in

#load json and create model
json_file = open(file_dir+'/results/DNN_model/%s_model.json'%spec,'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

#load weights into new model
loaded_model.load_weights(file_dir+'/results/DNN_model/%s_model.h5'%spec)
print('Loaded model from disk')

#compile model
loaded_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


mean_std=pd.read_csv(file_dir+'/results/DNN_model/%s_mean_std.txt'%spec,sep='\t')
mean_std=mean_std.to_numpy()



for i in range(1, len(pred_loc)):
    X=[]
    row, col = src.index(lon[i], lat[i])# spatial --> image coordinates  
    start=time.time()
    for j in range (1, 42):
        end=time.time()
        start1=time.time()
        band = src.read(j)
        end1 = time.time()
        start2=time.time()
        value = band[row,col]
        value = ((value - mean_std.item((j-1,1))) / mean_std.item((j-1,2))) #scale value
        X.append(value) 
        end2 = time.time()
    if all(i >= -1000 for i in X) == True:#exclude cells that still contain no-data values (probably bordering the sea)  
        startt=time.time()
        X=[X]
        X=np.vstack(X) 
        new_value = loaded_model.predict(x=X,batch_size=1,verbose=0) ###predict output value
        new_value=new_value.item((0,1))    
        #print('predicted prob. presence',new_value)
        new_band[row,col]=new_value  
        endd = time.time()
        print(start-end,start1-end1,start2-end2,startt-endd)
    else:
        new_band[row,col]=0


        
#write to file
with rasterio.open(file_dir+'/results/DNN_global_predictions/%s_predicted_map.tif'%spec, 'w', **profile) as dst:
    dst.write(new_band.astype(rasterio.float32), 1)



Loaded model from disk
-0.9177045822143555 -0.02299213409423828 0.0 -0.1629467010498047
-0.9607067108154297 -0.02297687530517578 0.0 -0.0010156631469726562
-0.9277012348175049 -0.023992538452148438 0.0 0.0
-0.928722620010376 -0.022994279861450195 -0.0010023117065429688 0.0


KeyboardInterrupt: 